### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### Timestamp data
Acquires timestamp and stores it

In [2]:
#Moved to API Call and further sections for more precise query purposes

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [3]:
from secrets import SecretKey
from secrets import AccessKey

In [4]:
from novadax import RequestClient as NovaClient

nova_client = NovaClient(AccessKey, SecretKey)
symbol = 'ETH_BRL'

### Importing tht List of currencies, making the API Call and the table Dataframe
List of crypto-currencies and table creation 

In [5]:
#API call
symbols = ['ETH_USDT', 'BTC_USDT']
symbol = 'ETH_USDT'
api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
drequest = requests.get(api_url).json()
print(drequest)

{'code': 'A10000', 'data': {'ask': '2000', 'baseVolume24h': '10.0814', 'bid': '1998', 'high24h': '2036.22', 'lastPrice': '1982.45', 'low24h': '1928.19', 'open24h': '1951.64', 'quoteVolume24h': '19907.8842', 'symbol': 'ETH_USDT', 'timestamp': 1613840497172}, 'message': 'Success'}


In [6]:
#Parse for data then get price
dfdata = drequest['data']
price = float(dfdata['lastPrice'])
baseVolume24h = dfdata['baseVolume24h']

In [14]:
#dolar price BRL: 1/(ETH/BRL / ETH/USD)
currency = 'R$'
dsymbol = 'ETH_BRL'
dsymbol_url = f'https://api.novadax.com/v1/market/ticker?symbol={dsymbol}'
xdrequest = requests.get(dsymbol_url).json()
dt = xdrequest['data']
dtprice = float(dt['lastPrice'])
dolar_calc = (dtprice / price)
dolar_price = 1/dolar_calc
print(currency, dolar_price)

R$ 5.032490237523734


In [15]:
#ballance
balance = nova_client.get_account_balance()
bdata = balance['data']
bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
bdataframe

,Currency,Balance
0,BTT,96.09424
1,USDT,0.09682579
2,ETH,0.0674948847
3,BRL,0.06
4,DGB,0.0353
5,LINK,0.0088601
6,DOGE,0.00881
7,XLM,0.00447
8,NULS,0.0044
9,IOTA,0.003905


In [16]:
#Dataframe get
#20210219 instructions: put the above wallet loop inside the below dataframe loop to call 'position' for each
#                       currency independently at time.
#r4f: https://stackoverflow.com/questions/16729574/how-to-get-a-value-from-a-cell-of-a-dataframe
#val = bdataframe['Balance'].values[1]
#print("U$DT", val)
my_columns = ['Symbol', 'Currency Price', 'Base Volume 24h', 'Number of Shares to Buy']
f_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
    fddata = requests.get(api_url).json()
    fdata = fddata['data']
    price = float(fdata['lastPrice'])
    baseVolume24h = fdata['baseVolume24h']
    dolarwal = bdataframe['Balance'].values[1]
    shares_to_buy = (float(dolarwal)/float(dolar_price)/float(price))
    for i in fdata['symbol'][:1]:
        f_dataframe.append(
            pd.Series(
                [
                    symbol,
                    price,
                    baseVolume24h,
                    'N/A'
                ]
                ,index = my_columns),
            ignore_index=True)
    
    
        f_dataframe = f_dataframe.append(
                                            pd.Series([symbol, 
                                                       fdata['lastPrice'], 

                                                       fdata['baseVolume24h'], 
                                                       shares_to_buy], 
                                                      index = my_columns), 
                                            ignore_index = True)

f_dataframe

,Symbol,Currency Price,Base Volume 24h,Number of Shares to Buy
0,ETH_USDT,1982.45,10.0814,9.705231e-06
1,BTC_USDT,56268.76,3.886683,3.419328e-07


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin.

In [17]:
#orders
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=10)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']

my_columns = ['Symbol', 'Ammount','Price']
f_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_trades(symbol, limit=5)
    f = 0
    for i in listtrades['data']:
        ldata = listtrades['data']
        lindice = ldata[f]
        amount = lindice['amount']
        price = float(lindice['price'])
        tsp = (lindice['timestamp'])
        f = f + 1
        f_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        f_dataframe = f_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                      ]
                ,index = my_columns), 
                ignore_index = True)

f_dataframe.sort_values('Price', ascending = True, inplace = True)
f_dataframe.reset_index(drop = True, inplace = True)
f_dataframe

,Symbol,Ammount,Price
0,ETH_USDT,0.0361,1978.23
1,ETH_USDT,0.0304,1982.45
2,ETH_USDT,0.024,1984.02
3,ETH_USDT,0.018,1991.00
4,ETH_USDT,0.0304,1993.01
5,BTC_USDT,0.000416,56188.76
6,BTC_USDT,0.002077,56268.76
7,BTC_USDT,0.018872,56368.17
8,BTC_USDT,0.490343,56401.09
9,BTC_USDT,0.207057,56401.09


In [12]:
#sort price [ok]
#wallet_amount [ok]
#buy lower(wallet_amount - amount)     
#sell + profit(constant*market momentum)
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}
#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [18]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
f_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [19]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [28]:
writer.save()